In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("../data/imageDataSet.csv")

In [3]:
df.head()

,1109,1111,1621,1623,2133,4693,4695,5193,5207,5319,...,23460,23484,23739,25230,25275,25742,26254,26556,27068,Target
0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,54.366787,0.000000,0.000000,...,97.608917,14.459713,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,2.201546,0.000000,0.585434,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1
2,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,25.582216,0.000000,0.000000,...,86.273682,17.683262,4.178553,13.121740,31.260714,3.847086,0.000000,0.0,0.000000,1
3,2.802592,0.000000,2.419251,0.0,25.850954,8.406624,3.889212,1.309759,14.579442,60.298489,...,155.205734,62.641945,0.000000,0.000000,1.351078,0.000000,0.000000,0.0,18.597864,1
4,13.896986,47.963455,17.366030,0.0,34.872738,18.890219,66.620743,0.000000,76.196136,90.715256,...,278.565826,110.468735,36.941467,124.530258,13.457292,123.339584,35.146267,0.0,0.000000,1


In [4]:
y = df['Target']
X = df.drop(['Target'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                    stratify=y)


In [5]:
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('LR', LogisticRegression(random_state=42))])


# source: https://towardsdatascience.com/a-simple-example-of-pipeline-in-machine-learning-with-scikit-learn-e726ffbb6976

In [6]:
hyperparameters = {'LR__penalty': ['l1', 'l2', 'elasticnet', 'none'],
                   'LR__solver': ['newton-cg','lbfgs','liblinear','sag','saga'],
                   'LR__multi_class': ['auto','ovr','multinomial'],
                   'LR__dual': [True, False],
                   'LR__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                   'LR__max_iter': np.arange(100, 500, 20).tolist()}

kfold = KFold(n_splits=5, random_state=42, shuffle=True)

gscv  = GridSearchCV(pipeline, 
                     param_grid=hyperparameters, 
                     cv=kfold, 
                     scoring='accuracy',
                     n_jobs=-1)

In [7]:
import json

gscv_result = gscv.fit(X_train, y_train)

print("GridSearchCV Best: ", gscv_result.best_score_)
print("Parameters:\n", json.dumps(gscv_result.best_params_, indent=2))

GridSearchCV Best:  0.8780119648293627
Parameters:
 {
  "LR__C": 10,
  "LR__dual": true,
  "LR__max_iter": 340,
  "LR__multi_class": "auto",
  "LR__penalty": "l2",
  "LR__solver": "liblinear"
}


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
